# Out of core computing

In this notebook we will demonstrate how to use `dask` to perform computation on a file of 2GB that cannot be fully loaded in memory (a Raspberry pi 3 has 1 GB of RAM)

## Create a h5py dataset

A `h5py` dataset references the data on disk without loading them to memory unless explicitely asked for.

In [2]:
import h5py
import os
f = h5py.File(os.path.join('data', 'random.hdf5'))
dset = f['/x']

In [ ]:
# Compute sum of large array, one million numbers at a time
sums = []
for i in range(0, 500000000, 1000000):
    chunk = dset[i: i + 1000000]  # pull out numpy array
    sums.append(chunk.sum())

total = sum(sums)
print(total)

## Compute the sum with `dask`

We can create a `dask` array from any object that presents the same interface as `numpy` arrays, in this case a `h5py` dataset. The chunk size is defining how big is each subsection of the array that is going to be loaded and manipulated by `dask`, many chunks can be loaded simultaneously in memory to make use of multiple cores.

In [3]:
import dask.array as da
x = da.from_array(dset, chunks=(1000000,))

`dask` computations are lazy, they are not evaluated immediately because `dask` can combine different operations together and optimize it computation.

In [16]:
result = x[:int(2e7)].sum()
result

dask.array<sum-agg..., shape=(), dtype=float32, chunksize=()>

In [17]:
%time result.compute()

CPU times: user 830 ms, sys: 1.57 s, total: 2.4 s
Wall time: 795 ms


20004710.0